In [11]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY
from gptchem.gpt_classifier import GPTClassifier
from gptchem.tuner import Tuner

from loguru import logger

logger.enable("gptchem")

In [30]:
mb = MatbenchBenchmark(
    autoload=True,
    subset=[
        "matbench_glass",
        "matbench_expt_is_metal",
    ],
)

2023-01-18 09:00:45 INFO     Initialized benchmark 'matbench_v0.1' with 2 tasks: 
['matbench_glass', 'matbench_expt_is_metal']


In [31]:
for task in mb.tasks:
    task.load()
    
    for fold_ind, fold in enumerate(task.folds):
        print(type(fold))
        classifier = GPTClassifier('glass', Tuner(n_epochs=8, learning_rate_multiplier=0.02, wandb_sync=False))
        train_inputs, train_outputs = task.get_train_and_val_data(fold)

        # train and validate your model
        classifier.fit(train_inputs, train_outputs.astype(int))

        # Get testing data
        test_inputs = task.get_test_data(fold, include_target=False)

        # Predict on the testing data
        # Your output should be a pandas series, numpy array, or python iterable
        # where the array elements are floats or bools
        predictions = classifier.predict(test_inputs)

        # Record your data!
        task.record(fold, predictions)

    print(f"{task.dataset_name}: Accuracy score {task.scores['accuracy']['mean']}")
    print(f"{task.dataset_name}: ROC score {task.scores['rocauc']['mean']}")

2023-01-18 09:00:45 INFO     Dataset matbench_glass already loaded; not reloading dataset.
<class 'int'>


Upload progress: 100%|██████████| 501k/501k [00:00<00:00, 799Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/03_classification/matbench/out/20230118_090045/train.jsonl: file-bPqgToXju1FuFr9LLqd3YhTH


2023-01-18 09:00:49.098 | DEBUG    | gptchem.tuner:tune:186 - Requested fine tuning. {
  "created_at": 1674028849,
  "events": [
    {
      "created_at": 1674028849,
      "level": "info",
      "message": "Created fine-tune: ft-Ot29PLMFNbmOUc1bvSzeifu1",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-Ot29PLMFNbmOUc1bvSzeifu1",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 500776,
      "created_at": 1674028848,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/03_classification/matbench/out/20230118_090045/train.jsonl",
      "id": "file-bPqgToXju1FuFr9LLqd3YhTH",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
   